In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

In [4]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


def gen_gpt_messages(prompt):
    messages = [{"role": "user", "content": prompt}]
    return messages


def get_completion(prompt, model="gpt-3.5-turbo", temperature = 0):
    response = client.chat.completions.create(
        model=model,
        messages=gen_gpt_messages(prompt),
        temperature=temperature,
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"

#### 分隔符分割输入

In [5]:
text = f"""
您应该提供尽可能清晰、具体的指示，以表达您希望模型执行的任务。\
这将引导模型朝向所需的输出，并降低收到无关或不正确响应的可能性。\
不要将写清晰的提示词与写简短的提示词混淆。\
在许多情况下，更长的提示词可以为模型提供更多的清晰度和上下文信息，从而导致更详细和相关的输出。
"""

# 需要总结的文本内容
prompt = f"""
把用三个反引号括起来的文本总结成一句话。
```{text}```
"""

# 指令内容，使用 ``` 来分隔指令和待总结的内容
response = get_completion(prompt)
print(response)

提供清晰、具体的指示可以引导模型朝向所需的输出，避免收到无关或不正确的响应。


#### 结构化输出

In [6]:
prompt = f"""
请生成包括书名、作者和类别的三本虚拟书籍清单，\
并以Json格式提供，其中包含以下键:book_id、title、author、genre。
"""
response = get_completion(prompt)
print(response)

{
    "books": [
        {
            "book_id": 1,
            "title": "The Great Gatsby",
            "author": "F. Scott Fitzgerald",
            "genre": "Fiction"
        },
        {
            "book_id": 2,
            "title": "To Kill a Mockingbird",
            "author": "Harper Lee",
            "genre": "Classic"
        },
        {
            "book_id": 3,
            "title": "The Da Vinci Code",
            "author": "Dan Brown",
            "genre": "Mystery"
        }
    ]
}


#### 模型检查是否满足条件

In [8]:
# 满足条件的输入（text中提供了步骤）
text_1 = f"""
泡一杯茶很容易。首先，需要把水烧开。\
在等待期间，拿一个杯子并把茶包放进去。\
一旦水足够热，就把它倒在茶包上。\
等待一会儿，让茶叶浸泡。几分钟后，取出茶包。\
如果您愿意，可以加一些糖或牛奶调味。\
就这样，您可以享受一杯美味的茶了。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Text 1 的总结:")
print(response)

Text 1 的总结:
第一步 - 把水烧开。
第二步 - 拿一个杯子并把茶包放进去。
第三步 - 把烧开的水倒在茶包上。
第四步 - 等待几分钟，让茶叶浸泡。
第五步 - 取出茶包。
第六步 - 如果愿意，可以加入糖或牛奶调味。
第七步 - 尽情享受您的茶！


In [9]:
# 不满足条件的输入（text中未提供预期指令）
text_2 = f"""
今天阳光明媚，鸟儿在歌唱。\
这是一个去公园散步的美好日子。\
鲜花盛开，树枝在微风中轻轻摇曳。\
人们外出享受着这美好的天气，有些人在野餐，有些人在玩游戏或者在草地上放松。\
这是一个完美的日子，可以在户外度过并欣赏大自然的美景。
"""
prompt = f"""
您将获得由三个引号括起来的文本。\
如果它包含一系列的指令，则需要按照以下格式重新编写这些指令：

第一步 - ...
第二步 - …
…
第N步 - …

如果文本中不包含一系列的指令，则直接写“未提供步骤”。"
\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Text 2 的总结:")
print(response)

Text 2 的总结:
未提供步骤。


#### few shot prompting少量成功示例

In [10]:
prompt = f"""
您的任务是以一致的风格回答问题。

<孩子>: 教我耐心。

<祖父母>: 挖出最深峡谷的河流源于一处不起眼的泉眼；最宏伟的交响乐从单一的音符开始；最复杂的挂毯以一根孤独的线开始编织。

<孩子>: 教我韧性。
"""
response = get_completion(prompt)
print(response)

<祖父母>: 就像大树在风雨中屹立不倒，韧性是一种坚韧不拔的品质。要学会在困难面前坚持不懈，永不放弃。记住，挫折和困难只会让你变得更加坚强。继续努力，坚持不懈，你就能拥有无坚不摧的韧性。


#### 指定完成一些任务

In [11]:
text = f"""
在一个迷人的村庄里，兄妹杰克和吉尔出发去一个山顶井里打水。\
他们一边唱着欢乐的歌，一边往上爬，\
然而不幸降临——杰克绊了一块石头，从山上滚了下来，吉尔紧随其后。\
虽然略有些摔伤，但他们还是回到了温馨的家中。\
尽管出了这样的意外，他们的冒险精神依然没有减弱，继续充满愉悦地探索。
"""
# example 1
prompt_1 = f"""
执行以下操作：
1-用一句话概括下面用三个反引号括起来的文本。
2-将摘要翻译成法语。
3-在法语摘要中列出每个人名。
4-输出一个 JSON 对象，其中包含以下键：French_summary，num_names。

请用换行符分隔您的答案。

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("prompt 1:")
print(response)

prompt 1:
1- 兄妹在山顶井里打水时发生意外，但他们的冒险精神依然不减。
2- Dans un charmant village, les frère et sœur Jack et Jill partent chercher de l'eau dans un puits au sommet de la montagne.
3- Jack, Jill
4- {
   "French_summary": "Dans un charmant village, les frère et sœur Jack et Jill partent chercher de l'eau dans un puits au sommet de la montagne.",
   "num_names": 2
}


In [13]:
#### 增加输出格式
prompt_2 = f"""
1-用一句话概括下面用<>括起来的文本。
2-将摘要翻译成英语。
3-在英语摘要中列出每个名称。
4-输出一个 JSON 对象，其中包含以下键：English_summary，num_names。

请使用以下格式：
文本：<要总结的文本>
摘要：<摘要>
翻译：<摘要的翻译>
名称：<英语摘要中的名称列表>
输出 JSON：<带有 English_summary 和 num_names 的 JSON>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nprompt 2:")
print(response)


prompt 2:
摘要：在一个迷人的村庄里，兄妹杰克和吉尔在去山顶井打水的途中遭遇意外，但他们的冒险精神依然坚定。

翻译：In a charming village, siblings Jack and Jill encounter an accident on their way to fetch water from a well on the mountaintop, but their adventurous spirit remains strong.

名称：Jack, Jill

输出 JSON：{"English_summary": "In a charming village, siblings Jack and Jill encounter an accident on their way to fetch water from a well on the mountaintop, but their adventurous spirit remains strong.", "num_names": 2}


#### 模型先解答，后评判

In [14]:
prompt = f"""
判断学生的解决方案是否正确。

问题:
我正在建造一个太阳能发电站，需要帮助计算财务。

    土地费用为 100美元/平方英尺
    我可以以 250美元/平方英尺的价格购买太阳能电池板
    我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
    作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：
设x为发电站的大小，单位为平方英尺。
费用：

    土地费用：100x
    太阳能电池板费用：250x
    维护费用：100,000美元+100x
    总费用：100x+250x+100,000美元+100x=450x+100,000美元
"""
response = get_completion(prompt)
print(response)


学生的解决方案是正确的。总费用的计算是正确的，即总费用为450x+100,000美元。


In [15]:
#增加输出格式
#要求模型先解答，后对比，最后给出测评
prompt = f"""
请判断学生的解决方案是否正确，请通过如下步骤解决这个问题：

步骤：
首先，自己解决问题，解决问题时列数学表达式。
然后将您的解决方案与学生的解决方案进行比较，并评估学生的解决方案是否正确。
在自己完成问题之前，请勿决定学生的解决方案是否正确。

使用以下格式：

问题：问题文本
学生的解决方案：学生的解决方案文本
实际解决方案和步骤：实际解决方案和步骤文本
**学生的计算结果：学生的计算结果文本
实际计算结果：实际计算结果文本
学生的计算结果和实际计算结果是否相同：是或否
学生的解决方案和实际解决方案是否相同：是或否**
学生的成绩：正确或不正确

问题：
我正在建造一个太阳能发电站，需要帮助计算财务。
- 土地费用为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判好了维护合同，每年需要支付固定的10万美元，并额外支付每平方英尺10美元
作为平方英尺数的函数，首年运营的总费用是多少。

学生的解决方案：

设x为发电站的大小，单位为平方英尺。
费用：
1. 土地费用：100x
2. 太阳能电池板费用：250x
3. 维护费用：100,000+100x
总费用：100x+250x+100,000+100x=450x+100,000

实际解决方案和步骤：
"""
response = get_completion(prompt)
print(response)

首年运营的总费用应该包括土地费用、太阳能电池板费用、维护费用和额外的维护费用。正确的计算步骤如下：

1. 土地费用：100美元/平方英尺 * x平方英尺
2. 太阳能电池板费用：250美元/平方英尺 * x平方英尺
3. 维护费用：10万美元 + 10美元/平方英尺 * x平方英尺
4. 总费用：土地费用 + 太阳能电池板费用 + 维护费用

学生的计算结果：450x + 100,000
实际计算结果：360x + 100,000
学生的计算结果和实际计算结果是否相同：否
学生的解决方案和实际解决方案是否相同：否

学生的成绩：不正确


#### 幻觉（模型编造出的回答）

In [17]:
prompt = f"""
给我一些研究LLM长度外推的论文，包括论文标题、主要内容和链接
"""

response = get_completion(prompt)
print(response)
# 若没搜索到，请给出我不知道，不要编造答案

1. 论文标题： "Extrapolating LLM Lengths: A Study on Predicting Long-term Memory Performance"
主要内容： 该论文研究了如何通过LLM长度外推来预测长期记忆性能。研究者通过分析不同LLM长度下的记忆表现，提出了一种预测模型，并对其进行了实验验证。
链接：https://www.researchgate.net/publication/123456789/Extrapolating_LLM_Lengths_A_Study_on_Predicting_Long-term_Memory_Performance

2. 论文标题： "Predicting Long-term Memory Performance using LLM Length Extrapolation Techniques"
主要内容： 该论文探讨了使用LLM长度外推技术来预测长期记忆性能的方法。研究者提出了一种基于机器学习的模型，通过训练数据来预测LLM长度与长期记忆表现之间的关系。
链接：https://www.sciencedirect.com/publication/123456789/Predicting_Long-term_Memory_Performance_using_LLM_Length_Extrapolation_Techniques

3. 论文标题： "A Study on Extrapolating LLM Lengths for Long-term Memory Enhancement"
主要内容： 该论文研究了如何通过LLM长度外推来增强长期记忆。研究者提出了一种基于神经网络的模型，通过优化LLM长度来提高长期记忆性能。
链接：https://www.frontiersin.org/publication/123456789/A_Study_on_Extrapolating_LLM_Lengths_for_Long-term_Memory_Enhancement


In [18]:
#如果你不知道答案，就说你不知道，不要试图编造答案。

prompt = f"""
给我一些研究LLM长度外推的论文，包括论文标题、主要内容和链接
如果你不知道答案，就说你不知道，不要试图编造答案。
"""

response = get_completion(prompt)
print(response)

抱歉，我不知道关于LLM长度外推的论文。您可以尝试在学术搜索引擎或学术数据库中查找相关论文。祝您好运！如果您有任何其他问题，我会尽力帮助您回答。
